Load Libraries

In [1]:
pip install sentence-transformers numpy python-dotenv chromadb groq pypdf


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 87.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.0/331.0 kB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 102.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 103.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from sentence_transformers import SentenceTransformer
from groq import Groq
import os
from pypdf import PdfReader
import chromadb
import numpy as np
from numpy import linalg

Load embedder model

In [3]:
model = SentenceTransformer('BAAI/bge-small-en-v1.5')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [18]:
# Normalize Embeddings

def normalize_embeddings(embeddings):
  norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
  return (embeddings/norms).tolist()

Function to extract text from documents

In [19]:
def extract_text(path):
  reader = PdfReader(path)
  text = ""
  for page in reader.pages:
    text += page.extract_text() + "\n"
  return text

Load the text the extract text

In [20]:
text = extract_text('/content/The_Constitution_of_Kenya_2010.pdf')

Turn into chunks

In [21]:
def chunk_text(text, size=300):
  words = text.split()
  chunks=[]
  for i in range(0, len(words), size):
    chunks.append(" ".join(words[i:i+size]))
  return chunks

In [22]:
chunks = chunk_text(text)

Turn text to chunks

In [16]:
const_embeddings = model.encode(chunks)

In [23]:
const_embeddings = normalize_embeddings(const_embeddings)
const_embeddings

[[-0.04393754526972771,
  0.013259949162602425,
  0.05073908343911171,
  0.0021900509018450975,
  0.01850125752389431,
  0.05690736323595047,
  0.06146608665585518,
  -0.049960196018218994,
  -0.02923876978456974,
  0.04982655495405197,
  -0.07521097362041473,
  0.015212235040962696,
  0.0001589200837770477,
  -0.003043261356651783,
  0.028794467449188232,
  -0.0017309869872406125,
  -0.009391095489263535,
  -0.015077725984156132,
  -0.08905229717493057,
  0.04579225927591324,
  0.10125244408845901,
  0.030525263398885727,
  -0.04197676479816437,
  0.024559535086154938,
  0.08305848389863968,
  0.04704508185386658,
  0.0228720735758543,
  0.020948195829987526,
  0.028590723872184753,
  -0.15442246198654175,
  -0.0664421021938324,
  -0.03275338560342789,
  0.013072249479591846,
  0.073114775121212,
  -0.02221520058810711,
  -0.01912190206348896,
  -0.021419474855065346,
  0.006115322932600975,
  -0.00597746018320322,
  0.048515137284994125,
  0.022882012650370598,
  0.04118728265166283,

Load chromadb to store documents and embeddings

In [24]:
client = chromadb.PersistentClient(path="./chroma_db")

In [25]:
collection = client.create_collection(
    name="cons_documents", metadata={"description": "My document collection"}
)

print("collection created:", collection.name)

collection created: cons_documents


In [26]:
const_embeddings = model.encode(chunks).tolist()

In [27]:
ids = [f"doc_{i}" for i in range(len(chunks))]

Add to collections

In [29]:
collection.add(
    documents=chunks,
    embeddings=const_embeddings,
    ids=ids
)

Add inferencing - The brain

In [33]:
import os
from groq import Groq

groq_client = Groq(api_key=("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"))

In [35]:
ai_model = 'meta-llama/llama-4-scout-17b-16e-instruct'

In [39]:
def generate_answer(question, retrieved_docs):
  context = "\n\n".join(retrieved_docs)

  system_prompt = """
  You are an expert assistant.
  Answer ONLY using the provided context.
  If the answer is not in the context, say:
  "The document does not contain this information"
  """

  user_prompt = f"""
  context:
  {context}

  Question:
  {question}
  """

  response = groq_client.chat.completions.create(
      model=ai_model,
      messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": user_prompt}
      ],
      temperature=0,
      max_tokens=800
  )

  return response.choices[0].message.content

In [40]:
def ask(question):
  query_embedding = model.encode([question])
  query_embedding = normalize_embeddings(query_embedding)

  results = collection.query(
      query_embeddings=query_embedding,
      n_results=3
  )

  retrieved_docs = results["documents"][0]

  return generate_answer(question, retrieved_docs)

In [44]:
answer = ask("What does the document say about lands")
print(answer)

The document classifies land into three categories: public, community, and private. 

- Public land is defined as land that was unalienated government land, land held by State organs, land transferred to the State, land with no individual or community ownership, land with no identifiable heir, minerals and mineral oils, government forests, game reserves, water catchment areas, national parks, animal sanctuaries, and specially protected areas, roads, and rivers, lakes, and other water bodies.
- Community land includes areas such as ancestral lands, lands traditionally occupied by hunter-gatherer communities, grazing areas, shrines, and trust land held by county governments.
- Private land consists of registered land held under freehold tenure, land held under leasehold tenure, and land declared private land under an Act of Parliament.

The document also provides that a person who is not a citizen may hold land on the basis of leasehold tenure only, and any such lease shall not exceed ni